In [1]:
from ultralytics import YOLO
import cv2
import math
import torch
import numpy as np

c:\Users\ditec\anaconda3\envs\recursion5\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
FIRE_WEIGHTS = "ML Models/Fire/fire.pt"
WEAPONS_WEIGHTS = "ML Models/weapons detection/best.pt"
VIOLENCE_WEIGHTS = "Violence Detection/ViolenceDet.pt"

In [6]:
classes = {
    "generic": ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ],
    "fire" : ["fire","smoke"],
    "violence":["Violence","Weapons"],
    "weapons" : ["Grenade","Handgun","Rifle","Steel arms"]
}

In [7]:
models = {}
models["generic"] = YOLO("yolov8n.pt")
models["fire"] = YOLO(FIRE_WEIGHTS)
models["weapons"] = YOLO(WEAPONS_WEIGHTS)

In [8]:
import time

In [9]:
import pandas as pd
import os

def create_csv():
  try:
    df = pd.read_csv('anomalies.csv')
  except FileNotFoundError:
    df = pd.DataFrame(columns=['Timestamp','Type of anomaly' ,'Number of People present', 'Path of image'])
    df.to_csv('anomalies.csv', index=False)
  return df


def append_to_csv(df, timestamp, num_inmates, image_path, typeofanomaly=""):
  new_data = {'Timestamp': timestamp, 'Type of anomaly':typeofanomaly, 'Number of People present': num_inmates, 'Path of image': image_path}
  new_data_df = pd.DataFrame.from_dict(new_data, orient='index').T
  df = pd.concat([df, new_data_df], ignore_index=True)
  df.to_csv('anomalies.csv', index=False)

inmatesthresholdtime = 2
anomaly_count = {} 
df = create_csv()

def process_model(model, img, classNames, model_name, numinmates , savetodir="frames"):
    global inmatesthresholdtime, anomaly_count, df
    
    results = model(img, stream=True)
    inmates = 0
    anomalies = []
    typeofanomaly = {
            "generic": "crowd",
            "fire": "Fire",
            "weapons": "Weapons"
        }
    for r in results:
        boxes = r.boxes
        for box in boxes:
            cls = int(box.cls[0])
            confidence = math.ceil((box.conf[0]*100))/100
            if confidence < 0.6:
                continue
            if cls >= len(classNames):
                print("Error: Class index out of range")
                continue
            accepted_list = ["person", "fire", "Grenade", "Handgun", "Rifle", "Steel arms","Violence","Weapons"]
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
            if model_name == "generic" and classNames[cls] == "person":
                inmates += 1
            if classNames[cls] in accepted_list:
                anomaly_key = (model_name, classNames[cls])
                anomaly_count[anomaly_key] = anomaly_count.get(anomaly_key, 0) + 1
                if anomaly_count[anomaly_key] >= 1:
                    timestamp = time.strftime('%Y-%m-%d %H:%M:%S') 
                    anomaly_text = f"{classNames[cls]} detected, storing the frame in the database as {savetodir}/{timestamp}.jpg at {timestamp}.\n"
                    print(anomaly_text)
                    anomalies.append(anomaly_text)
                    anomaly_count[anomaly_key] = 0
                    df = create_csv()
                    if model_name == "generic" and inmates > numinmates:
                      append_to_csv(df, timestamp, inmates, f"{savetodir}/{timestamp}.jpg", typeofanomaly[model_name])
                    elif model_name != "generic":
                      append_to_csv(df, timestamp, inmates, f"{savetodir}/{timestamp}.jpg", typeofanomaly[model_name])
                    with open("anomalies.txt", "a") as f:
                        f.write(anomaly_text)
                    # create the directory if it does not exist
                    if not os.path.exists(savetodir):
                        os.makedirs(savetodir)
                    cv2.imwrite(filename=savetodir+"/"+timestamp+".jpg", img=img)
                    
            print("Confidence --->", confidence)
            print("Class name -->", classNames[cls])
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2
            cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)

    if inmatesthresholdtime <= 0:
        inmatesthresholdtime = 2
    else:
        inmatesthresholdtime -= 1
    with open("anomalies.txt", "a") as f:
        f.write(" ".join(anomalies) )

     
def start_webcam(models, classes):
  cap = cv2.VideoCapture(0)
  while True:
    ret, frame = cap.read()
    if not ret:
      print("Error: Unable to capture frame")
      break
    for model_name, model in models.items():
      process_model(model, frame.copy(), classes[model_name], model_name, 10)  # Pass a copy of frame to avoid modification across models
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break

  cap.release()
  cv2.destroyAllWindows()

In [10]:
# run the function on demo video
def start_demo(models, classes):
    cap = cv2.VideoCapture("ML Models/weapons detection/gun.mp4")
    i = 0
    while True:
        i += 1
        print(f"{i}th frame")
        ret, frame = cap.read()
        if i%10 != 0:
            continue
        if not ret:
            print("Error: Unable to capture frame")
            break
        if cv2.waitKey(1) & 0xFF == ord('q'):
                break 
        for model_name, model in models.items():
            process_model(model, frame.copy(), classes[model_name], model_name, 10)  # Pass a copy of frame to avoid modification across models
            cv2.imshow("frame", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    cap.release()
    cv2.destroyAllWindows()

In [103]:
start_demo(models, classes)

1th frame
2th frame
3th frame
4th frame
5th frame
6th frame
7th frame
8th frame
9th frame
10th frame

0: 384x640 (no detections), 390.0ms
Speed: 101.6ms preprocess, 390.0ms inference, 14.0ms postprocess per image at shape (1, 3, 384, 640)

0: 480x800 (no detections), 486.5ms
Speed: 10.0ms preprocess, 486.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 800)

0: 256x448 (no detections), 491.8ms
Speed: 2.0ms preprocess, 491.8ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 448)
11th frame
12th frame
13th frame
14th frame
15th frame
16th frame
17th frame
18th frame
19th frame
20th frame

0: 384x640 (no detections), 102.0ms
Speed: 3.0ms preprocess, 102.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 480x800 (no detections), 308.0ms
Speed: 5.0ms preprocess, 308.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 800)

0: 256x448 (no detections), 259.0ms
Speed: 2.0ms preprocess, 259.0ms inference, 1.0ms postprocess per image a

In [26]:
start_webcam(models=models, classes=classes)


0: 480x640 1 person, 117.1ms
Confidence ---> 0.84
Class name --> person
Speed: 0.0ms preprocess, 117.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 608x800 1 default, 348.7ms
Confidence ---> 0.79
Class name --> smoke
Speed: 3.3ms preprocess, 348.7ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 800)

0: 352x448 1 Steel arms, 306.2ms
Confidence ---> 0.5
Class name --> Steel arms
Speed: 0.0ms preprocess, 306.2ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 448)

0: 480x640 1 person, 137.4ms
Confidence ---> 0.87
Class name --> person
Speed: 7.0ms preprocess, 137.4ms inference, 12.0ms postprocess per image at shape (1, 3, 480, 640)

0: 608x800 1 default, 400.5ms
Confidence ---> 0.75
Class name --> smoke
Speed: 0.0ms preprocess, 400.5ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 800)

0: 352x448 1 Steel arms, 271.4ms
Speed: 0.0ms preprocess, 271.4ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 448)

0: 4

In [11]:
import lida
from lida import llm, Manager, TextGenerationConfig
import pandas as pd
import os

In [12]:
os.environ["COHERE_API_KEY"] = "hk305qjjFaCIXME1dGS0OIIHDth4NFnTtuzpPCOg"

In [13]:
# Load your data (replace with your data source)
data = pd.read_csv("anomalies.csv")

lida = Manager(text_gen = llm("cohere")) 
textgen_config = TextGenerationConfig(n=3, temperature=0.5, use_cache=True)

try:
    summary = lida.summarize(data=data, summary_method="default", textgen_config=textgen_config)  
    goals = lida.goals(summary, n=2, textgen_config=textgen_config)
except Exception as e:
    print(e)

try: 
    for goal in goals:
        display(goal)
except Exception as e:
    print(e)

1 validation error for Goal
visualization
  Input should be a valid string [type=string_type, input_value={'type': 'histogram', 'op...ber of People present'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/string_type
name 'goals' is not defined


In [14]:

# textgen_config = TextGenerationConfig(n=1, temperature=0.2)
# persona = "I am a security guard at a prison. I want to be able to detect anomalies in the prison environment."
# personal_goals = lida.goals(summary, n=2, persona=persona, textgen_config=textgen_config)
# for goal in personal_goals:
#     display(goal)

i = 0
library = "matplotlib"

charts = lida.visualize(summary=summary, goal=goal, textgen_config=textgen_config, library=library)  

for chart in charts:
    try:
        chart.show()
    except Exception as e:
        print(e)

NameError: name 'goal' is not defined

In [57]:
import langchain
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import PromptTemplate
from langchain_community.llms.huggingface_hub import HuggingFaceHub
from langchain_community.chat_models import ChatHuggingFace
import os
from dotenv import load_dotenv, get_key

os.environ["HUGGINGFACEHUB_API_TOKEN"] = get_key(key_to_get="HUGGINGFACEHUB_API_KEY",dotenv_path=".env")

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.5,
        "repetition_penalty": 1.03,
    },
)

def chatwithbot(txt:str):
    prompt = PromptTemplate(template= "You're a helpful security assistant. You've been asked to help authorities. They want to be able to detect securty concerns in the city. They've provided you with a list of anomalies in the city with type of anomaly and timestamp.Answer questions of user from given data. Here's the data:\n{data}", input_variables=["data"])

    chat_model = ChatHuggingFace(llm=llm)
    # get the data from anomalies.txt
    with open("anomalies.txt", "r") as f:
        data = f.read()
    if len(data) == 0:
        data = "No anomalies detected"

    final_prompt = prompt.format(data=data)
    user_template= PromptTemplate(template="{user_input}", input_variables=["user_input"])

    messages = [
    SystemMessage(content=final_prompt),
    HumanMessage(content=user_template.format(user_input=txt))
    ]
    print(chat_model(messages))
    res = chat_model(messages).content
    res = res[res.find("<|assistant|>")+len("<|assistant|>"):]
    return res


In [58]:

def chat():
    while True:
        txt = input("Enter your message: ")
        if txt == "exit":
            break
        res = chatwithbot(txt)
        print(res)

In [59]:
chat()

content="<|system|>\nYou're a helpful security assistant. You've been asked to help authorities. They want to be able to detect securty concerns in the city. They've provided you with a list of anomalies in the city with type of anomaly and timestamp.Answer questions of user from given data. Here's the data:\nperson detected, storing the frame in the database as frames/2024-03-14 04:47:06.jpg at 2024-03-14 04:47:06.\nperson detected, storing the frame in the database as frames/2024-03-14 04:47:07.jpg at 2024-03-14 04:47:07.\nperson detected, storing the frame in the database as frames/2024-03-14 04:47:07.jpg at 2024-03-14 04:47:07.\nperson detected, storing the frame in the database as frames/2024-03-14 04:47:09.jpg at 2024-03-14 04:47:09.\nperson detected, storing the frame in the database as frames/2024-03-14 04:47:09.jpg at 2024-03-14 04:47:09.\nperson detected, storing the frame in the database as frames/2024-03-14 04:47:10.jpg at 2024-03-14 04:47:10.\nperson detected, storing the 